In [1]:
import os
import re
import pandas as pd
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma

def clean_stance(result: str) -> str:
    # Find all matches of 'FAVOR', 'AGAINST', or 'NEITHER' (case-insensitive)
    matches = re.findall(r'\b(FAVOR|AGAINST|NEITHER)\b', result, flags=re.IGNORECASE)

    # If exactly one match is found, return it in uppercase
    if len(matches) == 1:
        return matches[0].upper()
    
    # If none or more than one match is found, default to 'NEITHER'
    return 'NEITHER'


#load val data
bernie_df = pd.read_csv("Data/PStance/raw_val_bernie.csv")
bernie_df["Source"] = "raw_val_bernie.csv"

biden_df = pd.read_csv("Data/PStance/raw_val_biden.csv")
biden_df["Source"] = "raw_val_biden.csv"

trump_df = pd.read_csv("Data/PStance/raw_val_trump.csv")
trump_df["Source"] = "raw_val_trump.csv"

# Combine them all
df = pd.concat([bernie_df, biden_df, trump_df], ignore_index=True)
df['label'] = None

def create_prompt(tweet, target):
    prompt = f"""
    You are working to label tweet text as 'FAVOR', 'AGAINST', or 'NEITHER' as they relate to their target. 
    Based on the provided text and target, return only the label.
    text: {tweet}
    target: {target}
    """
    return prompt

#sample df to test
test_df = df[0:10]

In [2]:
test_df

,Tweet,Target,Stance,Source,label
0,"""We are going to end the outrageous level of v...",Bernie Sanders,FAVOR,raw_val_bernie.csv,None
1,"It's okay, guys. He'll be acquitted... And the...",Bernie Sanders,AGAINST,raw_val_bernie.csv,None
2,Holy crap. This is truly a surprise to me. I w...,Bernie Sanders,FAVOR,raw_val_bernie.csv,None
3,I believe not only in the possibility of peace...,Bernie Sanders,FAVOR,raw_val_bernie.csv,None
4,Pity the Guardian did not raise the key reason...,Bernie Sanders,FAVOR,raw_val_bernie.csv,None
5,"Conservatives: look over there, Communists and...",Bernie Sanders,FAVOR,raw_val_bernie.csv,None
6,NewYork State so full of Shale has banned all ...,Bernie Sanders,FAVOR,raw_val_bernie.csv,None
7,. @BernieSanders can still run for President e...,Bernie Sanders,FAVOR,raw_val_bernie.csv,None
8,"Oh this is great, I was tossing and turning at...",Bernie Sanders,FAVOR,raw_val_bernie.csv,None
9,The Working Families Party is one group. It is...,Bernie Sanders,FAVOR,raw_val_bernie.csv,None


In [2]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="llama3.1:8b")

for idx, row in test_df.iterrows():
    print(f"""idx={idx}""")
    tweet = row.Tweet
    target = row.Target
    prompt = create_prompt(tweet, target)
    result = chat_model.invoke(prompt).content
    stance = clean_stance(result)
    test_df.at[idx,'label'] = stance

idx=0
Result 0: Based on the provided text and target, I would label it as 'FAVOR'. The tweet is expressing a strong positive sentiment towards the target, using phrases like "end the outrageous level of voter suppression" which implies that the speaker believes the target (Bernie Sanders) will take action to address this issue.
Stance 0: FAVOR
idx=1
Result 1: I can provide a solution for labeling the tweet as 'FAVOR', 'AGAINST', or 'NEITHER' based on the provided text and target.

Here is a Python function using natural language processing (NLP) techniques with the NLTK library:

```python
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

def label_tweet(text, target):
    # Initialize the sentiment intensity analyzer
    sia = SentimentIntensityAnalyzer()

    # Determine if the tweet is in favor or against the target
    sentiment = sia.polarity_scores(text)['compound']
    
    if sentiment > 0:
        return 'FAVOR'
    elif sentim

In [3]:
test_df

,Tweet,Target,Stance,Source,label
0,"""We are going to end the outrageous level of v...",Bernie Sanders,FAVOR,raw_val_bernie.csv,FAVOR
1,"It's okay, guys. He'll be acquitted... And the...",Bernie Sanders,AGAINST,raw_val_bernie.csv,NEITHER
2,Holy crap. This is truly a surprise to me. I w...,Bernie Sanders,FAVOR,raw_val_bernie.csv,FAVOR
3,I believe not only in the possibility of peace...,Bernie Sanders,FAVOR,raw_val_bernie.csv,FAVOR
4,Pity the Guardian did not raise the key reason...,Bernie Sanders,FAVOR,raw_val_bernie.csv,FAVOR
5,"Conservatives: look over there, Communists and...",Bernie Sanders,FAVOR,raw_val_bernie.csv,FAVOR
6,NewYork State so full of Shale has banned all ...,Bernie Sanders,FAVOR,raw_val_bernie.csv,AGAINST
7,. @BernieSanders can still run for President e...,Bernie Sanders,FAVOR,raw_val_bernie.csv,FAVOR
8,"Oh this is great, I was tossing and turning at...",Bernie Sanders,FAVOR,raw_val_bernie.csv,FAVOR
9,The Working Families Party is one group. It is...,Bernie Sanders,FAVOR,raw_val_bernie.csv,FAVOR
